In [1]:
"""
This section imports the necessary libraries.
qiskit is the main library for quantum computing,
AerSimulator is used for simulating quantum circuits,
and numpy is used for numerical operations like generating random numbers.
"""

from qiskit import QuantumCircuit
from math import ceil, floor
import pandas as pd
from qiskit_ibm_provider import IBMProvider
import time

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
provider = IBMProvider()

backend = provider.get_backend('ibmq_qasm_simulator')

In [3]:
class QuantumTeleportation:
    """
    This class achieves quantum teleportation of either the |0> state or the
    |1> state. It depends on the number of qubits used for the teleportation;
    if 2 qubits are used then the teleportation is done using Bell states, if
    more than 2 qubits are used then the teleportation is done using GHZ states.
    """
    def __init__(self, qubits=2, initial_state=0, teleportation_state='00', backend=backend):
        """
        This function initializes the entire class.
        
        Parameters:
        --------------
        
        qubits :              The number of qubits used for the teleportation
        
        initial_state :       The state that will be teleported
        
        teleportation_state : The entangled state that will be used to achieve
                              the teleportation
        """
        # Making the number of qubits a property of self
        self.M = qubits
        
        # Creating a quantum circuit with M+1 qubits and M+1 classical bits
        self.circuit = QuantumCircuit(self.M+1, self.M+1)
        
        # Establishing the state we will be teleporting
        self.initial_state = initial_state  # (either 0 or 1)
        
        # Establishing the state we will use for teleportation
        self.teleportation_state = teleportation_state  # (either '00', '01', '10', or '11')
        
        # Set the backend
        self.backend = backend

    def initialize_state(self):
        """
        This function modifies the state to be teleported, if necessary.
        """
        if self.initial_state == 1:
            self.circuit.x(0)  # Apply X gate to qubit 0 if the state to teleport is 1
        
    def create_entangled_state(self):
        """
        This function creates the entanglement between the last M-1 qubits.
        """
        # We first consider which teleportation state we will be using, these are
        # the 'beta' states
        if self.teleportation_state[0] == '1':
            self.circuit.x(1) # If x = 1, we change qubit 1 to the |1> state
            
        if self.teleportation_state[1] == '1':
            for i in range(floor(self.M/2)):
                self.circuit.x(i+1+ceil(self.M/2)) # If y = 1, we change the last floor(M/2) qubits to the |1> state
        
        # Performing the steps necessary for entanglement
        self.circuit.h(1) # First apply the Hadamard gate to qubit 1
        for qubit in range(self.M-1):
            self.circuit.cx(1, qubit+2) # Then apply a C-Not gate between qubit 1 and all the remaining qubits
            
    def teleportation_protocol(self):
        """
        This function performs the teleportation protocol.
        """
        # Performing the steps necessary for teleportation
        for qubit in range(self.M-1):
            self.circuit.cx(0, qubit+1) # First applying a C-Not gate between qubit 0 and all the remaining qubits (except qubit M+1)
        self.circuit.h(0) # The we apply the Hadamard gate to qubit 0
        
        # Finalizing the teleportation protocol for when y = 1 and M < 4
        if self.teleportation_state[1] == '1' and self.M < 4:
            for i in range(floor(self.M/2)):
                self.circuit.x(i+ceil(self.M/2)) # Applying additional x-gates to account for initial teleportation state
        
        # Measuring all qubits into classical bits (except for qubit M+1)
        qubit_list = list(range(0, self.M))
        self.circuit.measure(qubit_list, qubit_list)
        
        # Making the necessary adjustments to qubit M+1 based on measurement outcomes
        self.circuit.cx(self.M-1, self.M)
        self.circuit.cz(0, self.M)
        
        # Finally measuring qubit M+1
        self.circuit.measure(self.M, self.M)

    def execute_circuit(self):
        """
        This function executes the entire circuit built by the previous functions,
        then runs it on the backend.
        """
        if self.backend is None:
            raise ValueError("Backend must be set before executing the circuit.")
        
        # Calling all functions from above
        self.initialize_state()
        self.create_entangled_state()
        self.teleportation_protocol()
        
        # Running the circuit on the backend
        result = self.backend.run(self.circuit, shots=1000).result()
        
        # Getting the measurement outcomes from the circuit
        counts = result.get_counts(self.circuit)
        
        return counts # Return the counts dictionary

In [4]:
# M qubits in the teleportation, M+1 qubits total
M = 30

# Initial state is either a 0 or a 1
initial_states = [0, 1]

# Listing all different entanglement states
beta_states = ['00', '10', '01', '11']
qubits = list(range(2, M+1))

count_results = {}
teleportation_results = {}

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(f"Starting: {current_time}")

for qubit in qubits:
    for initial in initial_states:
        for beta in beta_states:
            # Create a QuantumTeleportation object for each combination
            teleport = QuantumTeleportation(qubits=qubit, initial_state=initial, teleportation_state=beta)
            
            # Execute the circuit and get counts
            counts = teleport.execute_circuit()
            
            # Convert initial_state to a string ('0' or '1') for comparison
            initial_state_str = str(teleport.initial_state)
            
            # Check if all counts begin with the same value as the initial_state
            successful_teleportation = all(measurement.startswith(initial_state_str) for measurement in counts)
            
            # Store the result in the dictionary
            count_results[(qubit, initial, beta)] = counts
            teleportation_results[(qubit, initial, beta)] = successful_teleportation

    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    print(f"Update: Completed {qubit} qubits at {current_time}")
        
current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print(f"Finished job: {current_time}")

Starting: 2024-04-19 04:33:34
Update: Completed 2 qubits at 2024-04-19 04:34:21
Update: Completed 3 qubits at 2024-04-19 04:34:58
Update: Completed 4 qubits at 2024-04-19 04:35:45
Update: Completed 5 qubits at 2024-04-19 04:36:28
Update: Completed 6 qubits at 2024-04-19 04:37:12
Update: Completed 7 qubits at 2024-04-19 04:37:48
Update: Completed 8 qubits at 2024-04-19 04:38:20
Update: Completed 9 qubits at 2024-04-19 04:38:56
Update: Completed 10 qubits at 2024-04-19 04:39:34
Update: Completed 11 qubits at 2024-04-19 04:40:09
Update: Completed 12 qubits at 2024-04-19 04:40:47
Update: Completed 13 qubits at 2024-04-19 04:41:22
Update: Completed 14 qubits at 2024-04-19 04:41:56
Update: Completed 15 qubits at 2024-04-19 04:42:34
Update: Completed 16 qubits at 2024-04-19 04:43:12
Update: Completed 17 qubits at 2024-04-19 04:43:44
Update: Completed 18 qubits at 2024-04-19 04:44:21
Update: Completed 19 qubits at 2024-04-19 04:44:55
Update: Completed 20 qubits at 2024-04-19 04:45:34
Update: C

In [5]:
# Converting the dictionary to a pandas DataFrame for better visualization and analysis
df_results = pd.DataFrame(list(teleportation_results.items()), columns=['Parameters', 'Success'])
df_results['Qubits'] = df_results['Parameters'].apply(lambda x: x[0])
df_results['Initial_State'] = df_results['Parameters'].apply(lambda x: x[1])
df_results['Beta_State'] = df_results['Parameters'].apply(lambda x: x[2])
df_results.drop('Parameters', axis=1, inplace=True)

# To save the DataFrame to a CSV file
df_results.to_csv('teleportation_results_simulator.csv', index=False)

In [6]:
# Convert results dictionary to a list of tuples for DataFrame creation
data = []
for (qubits, initial_state, beta_state), counts in count_results.items():
    for measurement_result, frequency in counts.items():
        data.append({
            'Qubits': qubits,
            'Initial State': initial_state,
            'Beta State': beta_state,
            'Measurement Result': measurement_result,
            'Frequency': frequency
        })

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('teleportation_results_detailed_simulator.csv', index=False)